# Spectral Clustering  

In this notebook, we will use cuGraph to identify the cluster in a test graph using the Spectral Clustering Balance Cut algorith,  

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 03/08/2019

RAPIDS Versions: 0.6.0

Test Hardware
*GP100 32G, CUDA 9,2




Spectral clustering uses the eigenvectors of a Laplacian of the input graph to find a given number of clusters which satisfy a given quality metric. Balanced Cut is such a quality metric. 

@See:  https://en.wikipedia.org/wiki/Spectral_clustering

To perform spectral clustering using the balanced cut metric in cugraph use:
__cugraph.spectralBalancedCutClustering(G, num_clusters, num_eigen_vects)__

Input
* __G__: A cugraph.Graph object
* __num_clusters__: The number of clusters to find
* __num_eig__: (optional) The number of eigenvectors to use

Returns
* __df__: cudf.DataFrame with two names columns:
    * df["vertex"]: The vertex id.
    * df["cluster"]: The assigned partition.


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

Zachary used a min-cut flow model to partition the graph into two clusters, shown by the circles and squares.  Zarchary wanted just two cluster based on a conflict that caused the Karate club to break into two separate clubs.  Many social network clustering methods identify more that two social groups in the data.

In [ ]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from scipy.io import mmread

In [ ]:
# Read in the data file into scipy matrix format
mmFile='../data/networks/karate.mtx'
M = mmread(mmFile).asfptype().tolil()
M = M.tocsr()

In [ ]:
# Load the structure of the graph into GPU memory and create a CuGraph
# graph object:
row_offsets = cudf.Series(M.indptr)
col_indices = cudf.Series(M.indices)
values = cudf.Series(M.data)
G = cugraph.Graph()
G.add_adj_list(row_offsets, col_indices, values)

In [ ]:
# Call spectralBalancedCutClustering on the graph for 3 clusters
# using 3 eigenvectors:
df = cugraph.spectralBalancedCutClustering(G, 3, num_eigen_vects=3)

In [ ]:
df.dtypes

In [ ]:
# Check the edge cut score for the produced clustering
score = cugraph.analyzeClustering_edge_cut(G, 2, df['cluster'])
score

In [ ]:
def print_cluster(df, id):
    
    # See which nodes are in cluster 0:
    part = []
    for i in range(len(df)):
        if (df['cluster'][i] == id):
            part.append(df['vertex'][i])
    print(part)

In [ ]:
# See which nodes are in cluster 0:
print_cluster(df, 0)

In [ ]:
# See which nodes are in cluster 1:
print_cluster(df, 1)

In [ ]:
# See which nodes are in cluster 2:
print_cluster(df, 2)

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___